In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'co2': 0.0008}
band = [11]
atmpro = trp
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [3, 3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (50, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.66, 1.66, 1.75), (1.75, 1.6, 1.85)]
  wgt = [(0.7, 0.8, 0.7), (0.8, 0.7, 0.8)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.184672                    1  0.434244
0.000750        2  0.016288                    2  0.039704
0.001052        3  0.016674                    3  0.007065
0.001476        4  0.011332                    4 -0.013964
0.002070        5  0.005571                    5 -0.019929
0.002904        6 -0.000089                    6 -0.023342
0.004074        7 -0.005882                    7 -0.021334
0.005714        8 -0.011988                    8 -0.019909
0.008015        9 -0.018318                    9 -0.019735
0.011243       10 -0.024791                   10 -0.019811
0.015771       11 -0.031406                   11 -0.023091
0.022122       12 -0.038466                   12 -0.028426
0.031031       13 -0.047463                   13 -0.039806
0.043528       14 -0.063877                   14 -0.060503
0.061057       15 -0.075168                   15 -0.072157
0.085645       16 -0.072365                   16 -0.060737
0.120136       17 -0.064187                   17 -0.039507
0.168516       18 -0.052885                   18 -0.022557
0.236378       19 -0.032057                   19 -0.009449
0.331549       20  0.005494                   20  0.010220
0.465100       21  0.082068                   21  0.068487
0.652400       22  0.239684                   22  0.216115
0.915100       23  0.281989                   23  0.271148
1.283650       24  0.200928                   24  0.227111
1.800600       25  0.139208                   25  0.182650
2.525700       26  0.081112                   26  0.115009
3.542800       27  0.046390                   27  0.055724
4.969550       28  0.026021                   28  0.017695
6.970850       29  0.013057                   29  0.008268
9.778100       30  0.005086                   30  0.009783
13.715850      31  0.000732                   31  0.007239
19.239350      32 -0.001099                   32  0.001999
26.987250      33 -0.001510                   33 -0.001620
37.855300      34 -0.001338                   34 -0.001646
53.100050      35 -0.002079                   35 -0.001859
73.887500      36 -0.002799                   36 -0.002599
97.662500      37 -0.003241                   37 -0.003183
121.437500     38 -0.003610                   38 -0.003589
145.212500     39 -0.003701                   39 -0.003668
168.987500     40 -0.004033                   40 -0.004096
192.762500     41 -0.004349                   41 -0.004442
216.537500     42 -0.004628                   42 -0.004753
240.312500     43 -0.004861                   43 -0.004951
264.087500     44 -0.005048                   44 -0.005086
287.862500     45 -0.005187                   45 -0.005133
311.637500     46 -0.005278                   46 -0.005064
335.412500     47 -0.005321                   47 -0.005011
359.187500     48 -0.005314                   48 -0.004796
382.962500     49 -0.005256                   49 -0.004596
406.737500     50 -0.005145                   50 -0.004336
430.512500     51 -0.004981                   51 -0.004012
454.287500     52 -0.004763                   52 -0.003797
478.062500     53 -0.004491                   53 -0.003326
501.837500     54 -0.004161                   54 -0.003118
525.612500     55 -0.003768                   55 -0.002530
549.387500     56 -0.003311                   56 -0.002215
573.162500     57 -0.002779                   57 -0.001566
596.937500     58 -0.002151                   58 -0.000966
620.712500     59 -0.001390                   59 -0.000168
644.487500     60 -0.000403                   60  0.000896
668.262500     61  0.000898                   61  0.002278
692.037500     62  0.002346                   62  0.003756
715.812500     63  0.003278                   63  0.004322
739.587500     64  0.003231                   64  0.004022
763.362500     65  0.002607                   65  0.002774
78

# Fluxes by Layer

CRD                        CLIRAD                \
                       flug      fldg     fnetg      flug          fldg   
pressure    level                                                         
0.000000    1     -6.497105  0.000000 -6.497105 -6.405478  8.551266e-09   
0.000624    2     -6.497111  0.000019 -6.497091 -6.405485  3.949445e-05   
0.000876    3     -6.497113  0.000022 -6.497091 -6.405489  4.432277e-05   
0.001229    4     -6.497117  0.000027 -6.497090 -6.405493  4.928118e-05   
0.001723    5     -6.497121  0.000032 -6.497089 -6.405500  5.455145e-05   
0.002417    6     -6.497127  0.000038 -6.497089 -6.405507  6.077444e-05   
0.003391    7     -6.497135  0.000046 -6.497089 -6.405518  6.855457e-05   
0.004757    8     -6.497146  0.000056 -6.497090 -6.405532  7.897058e-05   
0.006672    9     -6.497161  0.000068 -6.497093 -6.405550  9.301899e-05   
0.009359    10    -6.497181  0.000082 -6.497099 -6.405576  1.121200e-04   
0.013128    11    -6.497209  0.000099 -6.497110 -6.405611  1.383228e-04   
0.018415    12    -6.497248  0.000119 -6.497129 -6.405661  1.737487e-04   
0.025830    13    -6.497305  0.000142 -6.497163 -6.405733  2.212436e-04   
0.036232    14    -6.497391  0.000170 -6.497222 -6.405844  2.827438e-04   
0.050823    15    -6.497536  0.000204 -6.497332 -6.406028  3.619418e-04   
0.071291    16    -6.497785  0.000271 -6.497514 -6.406331  4.902502e-04   
0.100000    17    -6.498188  0.000428 -6.497760 -6.406793  7.461415e-04   
0.140271    18    -6.498810  0.000743 -6.498067 -6.407465  1.229517e-03   
0.196760    19    -6.499753  0.001332 -6.498420 -6.408428  2.041852e-03   
0.275997    20    -6.501141  0.002420 -6.498721 -6.409785  3.309762e-03   
0.387100    21    -6.503066  0.004417 -6.498649 -6.411608  5.267447e-03   
0.543100    22    -6.505200  0.008068 -6.497132 -6.413539  8.464462e-03   
0.761700    23    -6.505213  0.014287 -6.490926 -6.413232  1.375381e-02   
1.068500    24    -6.501854  0.021176 -6.480678 -6.409740  2.011754e-02   
1.498800    25    -6.496691  0.026256 -6.470436 -6.404556  2.651035e-02   
2.102400    26    -6.490291  0.029809 -6.460482 -6.398114  3.312908e-02   
2.949000    27    -6.483833  0.031485 -6.452348 -6.391414  3.796435e-02   
4.136600    28    -6.477709  0.031887 -6.445822 -6.384721  3.911215e-02   
5.802500    29    -6.471981  0.031294 -6.440687 -6.378159  3.604190e-02   
8.139200    30    -6.466744  0.029671 -6.437073 -6.371803  3.197433e-02   
11.417000   31    -6.462123  0.027025 -6.435098 -6.366001  2.997136e-02   
16.014700   32    -6.458265  0.023566 -6.434699 -6.360893  2.880701e-02   
22.464000   33    -6.455330  0.019791 -6.435539 -6.356574  2.601427e-02   
31.510500   34    -6.453441  0.016284 -6.437157 -6.353119  2.082391e-02   
44.200100   35    -6.452401  0.013232 -6.439168 -6.350143  1.537290e-02   
62.000000   36    -6.453467  0.009915 -6.443552 -6.349335  1.064392e-02   
85.775000   37    -6.458329  0.006894 -6.451435 -6.353129  7.116650e-03   
109.550000  38    -6.466062  0.005501 -6.460561 -6.360817  5.839179e-03   
133.325000  39    -6.476813  0.006085 -6.470729 -6.372178  7.090933e-03   
157.100000  40    -6.490060  0.008909 -6.481151 -6.386204  1.078590e-02   
180.875000  41    -6.505906  0.013396 -6.492509 -6.402959  1.600420e-02   
204.650000  42    -6.524433  0.019676 -6.504757 -6.422479  2.301244e-02   
228.425000  43    -6.545701  0.027910 -6.517791 -6.444914  3.206005e-02   
252.200000  44    -6.569735  0.038254 -6.531481 -6.470227  4.342773e-02   
275.975000  45    -6.596550  0.050852 -6.545698 -6.498461  5.733666e-02   
299.750000  46    -6.626143  0.065836 -6.560307 -6.529624  7.404321e-02   
323.525000  47    -6.658499  0.083326 -6.575173 -6.563573  9.373038e-02   
347.300000  48    -6.693588  0.103431 -6.590158 -6.600471  1.165148e-01   
371.075000  49    -6.731370  0.126248 -6.605122 -6.640138  1.426737e-01   
394.850000  50    -6.771790  0.151865 -6.619924 -6.682557  1.721479e-01   
418.625000  51    -6.814776  0.180363 -6.634414 -

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')